## Sales Forecasting by Product and Market 

# Index

<a href='#Overview'>Source Data'</a>
- a. Selection criteria to run SAP BI extract
- b. Layout of BI extract.
- c. Overview of the data


<a href='#Preprocessing'>Preprocessing'</a>
- 1. Ensure the year is 4 digits long
- 2. Ensure only the data for the relevant scope is kept; the rest is dropped
- 3. Format the value columns into a float¶
- 4. Identify entity and market to be analysed
- 5. Identify the data's datetime and remove 2022
- 6. Ensure all months, in the analysis windows, are included in the dataframe
- 7. Fill missing YTD values
- 8. Identify monthly movements¶
- 9. Data Analysis of the whole TimeSeries
        - a. Descriptive Statistics
        - b. Visual Analysis - Original, YTD and Monthly Mov
        - c. Visual analysis - monthly movements only
- 10. Address outliers



<a href='#Stationarity'>Stationarity'</a>
- 11 . Serie Stationarity
    - a. Decomposition
    - b. Differencing to make the series stationary
    - c. Testing stationarity (ADF test)
    - d. Copies of df for alternative modelling approaches
    - e. Identify train and validation set

<a href='#Parameters'>Arima's parameters'</a>
- 21. ARIMA approach (over all time series) - Find p & q parameters' values
        - a. PACF
        - b. ACF
        - c. Find the values of p and q for the ARIMA models
        - d . Analysis of residuals from the AR model

<a href='#Arima'>ARIMA's models'</a>
- 22. Forecasting AR, ARMA, SARIMAX Models
        - a. Identify train and validation set
        - b. Fit the data
            - Model no 1 - AR model
            - Model no 2 - ARIMA model (taken from ARIMA class where d=0)
            - Model no 3 - SARIMAX Model
        - c. Analysis of residual


<a href='#Prophet'>Prophet Model</a>
- 23. Forecasting Prophet Model 

<a href='#NN'>Neural Network Model'</a>
- 24. Neural Network

<a href='#XG'>XGBoost Model'</a>
- 25. Forecasting XGBoost

<a href='#HW'>Holt-Wilson Model'</a>
- 26. Forecasting Holt-Wilson Mode

<a href='#Darts'>Darts Model'</a>
- 27. Forecasting Darts Model

<a href='#MAE'>Metrics'</a>
- 30. Models' MAE comparison
- 31. Models' MAPE comparison
- 32. Models' RSME comparison
- 33. Models' MASE comparison
- 34. Metrics Summary
- 35. Selection of best ML model
- 36. Notes about Metrics

<a href='#FOR'>Bring forecasts back to original scale'</a>
- 40. Monthly Forecasts
- 41. YTD Forecasts

<a href='#Hyper'>Appendix  - Sarimax Model optimization '</a>
- 50. HyperParameters Optimization (with fixed train/ test split)
- 51. Cross Validation (with fixed order and seasonal order)
- 52. Cross Validation and HyperParameters Optimization (with variable train/ test split and variable order and seasonal order)

In [1]:
import re
import pandas as pd
import numpy as np

from numpy import mean
from numpy import std
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime
from scipy.stats import shapiro


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sktime.performance_metrics.forecasting import MeanAbsoluteScaledError
from statsmodels.api import tsa

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import ar_select_order
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from xgboost import XGBRegressor

from statsmodels.tsa.holtwinters import SimpleExpSmoothing   
from statsmodels.tsa.holtwinters import ExponentialSmoothing

import darts
from darts import TimeSeries
from darts.models import ExponentialSmoothing as ES
from darts.metrics import mape

from typing import *
from dateutil.relativedelta import relativedelta

import itertools
import warnings
warnings.filterwarnings("ignore") # specify to ignore warning messages

import time
import pickle

C:\Anaconda3\lib\site-packages\dask\dataframe\utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
t_start = time.time()

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', None)


In [ ]:
import logging
logging.getLogger('fbprophet').setLevel(logging.WARNING) 

<a id='Overview'></a>

# Source Data

### a.  Selection criteria to run SAP BI extract

### b. Layout of BI extract.

In [ ]:
# load source file 
data = pd.read_csv('SalesFile.csv', skiprows=3, encoding= 'unicode_escape') 
data.info()

In [ ]:
# rename columns
data=data.rename(columns={'Account Row':'Acct','Unnamed: 1':'Acct_description','Company code':'Entity','Data Entry point':'DEP','Audit ID':'Au_Id','Unnamed: 3':'Product_Description', 'Unnamed: 5':'Mkt_Description','Unnamed: 8':'AU_Description','Unnamed: 10':'Entity_Description','Unnamed: 13':'Prod_Description','£':'Value','Year/Period':'YrPd'})
data.head(2)

### c. Overview of the data

In [ ]:
# Quick overview of dataset's data type 
data.info()

<a id='Preprocessing'></a>

# Preprocessing

###  1. Ensure the year is 4 digits long

In [ ]:
# ensure the year is 4 digits long
'''
A: YrPd is converted into a string so that we can use RE to retrieve those entries with a 3 digits year. 
Once identified, a zero is added to these offensive years. Years in the correct format are left untouched.

'''

data['YrPd']=data['YrPd'].astype(str)
y=r"^(\d*\.\d{3})$"  
def fix_year(data):
    """ A function to ensure year is 4 digits long """    
    x=re.findall(y,str(data))
    if x:
        return str(data)+'0'
    else:
        return data
    
data['YrPd']=data['YrPd'].apply(fix_year)
#data.tail(1)

### 2. Ensure only the data for the relevant scope is kept; the rest is dropped

In [ ]:
# keep only the relevant scope
''' 
A: Only the data where the Scope matches the DEP will be kept. 
In details: Only data where 2 last 2 characters of the Scope match the last 2 digits of the year in the DEP will be kept. 
The rest will be removed.

'''
def final_scope(data):
    '''A function retain data where the scope matches the year '''
    if data.loc['YrPd'][-2:]== data.loc['Scope'][-2:]:
        return 1
    else:
        return 0
data['Final_Scope']=data.apply(final_scope,axis=1)
data=data[data['Final_Scope']==1]
df=data.copy()


In [ ]:
# Features no longer needed are removed.
df.drop(['Version','Rate','Au_Id','Scope','Final_Scope'],axis=1, inplace=True)


In [ ]:
df.tail(1)

In [ ]:
# view information about the data in the dataset
#df.info()

In [ ]:
# view size (no of rowsa and number of columns) of the dataset
df.shape

### 3. format the value column into a float 

In [ ]:
#  format the value column into a float
'''
T: The Value data needs to be formatted as a float
'''
df['Value']=df['Value'].str.replace(',','')
df['Value']=df['Value'].str.strip(')').str.replace('\(','-').astype(float)
df['Value']=df['Value']*-1
#df.head(10)

In [ ]:
df.head(2)

### 4. Identify entity and market to be analysed

In [ ]:
# find no of datapoints by products and entities 
subset=df.groupby(['Product','Entity']).agg(date_min=('DEP','min'), date_max=('DEP','max'),
                                              size=('Product','size'), value_min=('Value','min'),
                                              value_max=('Value','max')).sort_values(by=['size','value_max'],ascending=False)
subset

In [ ]:
# extract the product and entity with the heighest no of datapoints.
extract=subset.iloc[0,:]
extract

In [ ]:
# filter only datapoints with the chose product and entity
'''
All values below should say 1 a part from:
DEP, 
scope (no of years in consideration +1), 
YrPd (should be the same to DEP; if less it means there are null Value values) and 
Value 
'''
prod=extract.name[0]
ent=extract.name[1]
noDpt=extract.size
print( 'Product chosen: ',prod, '  Entity chosen: ', ent, '  Lowest DEP: ', 
      extract.date_min, '  Highest DEP: ', extract.date_max )
df=df[(df['Product']==prod) & (df['Entity']==ent)]
df.nunique()

Product 499 = Dental hygiene related product

### 5. Identify the data's datetime and remove 2022 data

In [ ]:
# set datetime index for the series
'''
A: Use the existing YrPd feature to create a list comprehension to retrieve the month and year and generate a datetime feature with this format: yyyy-mm-dd.
Set this new feature as the index of the time series.

'''
df['Date'] = [datetime(day=1, month=int(s[0]), year=int(s[-4:]))  
              if len(s)==6 else datetime(day=1,month=int(s[:2]), year=int(s[-4:]) ) 
              for s in df['YrPd']]

In [ ]:
#remove 2022 data from the dataset
'''
A: drop the first 6 months of 2022 from the dataset so we have data for full years
'''
df=df[df['Date']<='2021-12-01']

In [ ]:
df.set_index('Date', inplace=True)
df=df['Value']
# attach a data frequency to the date column
df.index = pd.DatetimeIndex(df.index.values,
                               freq=df.index.inferred_freq)
df.name = 'Original_values'
df.tail(5)

### 6. Ensure all months, in the analysis windows, are included in the timeframe

In [ ]:
# ensure all timespoints are in the series with no gaps
''' 
T: Ensure the timeseries has all the datapoint expected according to the frequency of the data collection 
(All months and years, within the timeframe under analysis, need to be present in the dataframe's index) 
A: Generate a complete Timestamp object, for the duration of all the expected timeserie, and concatenate 
the complete Timestamp object with the timeseries to see where missing data is

'''

max_val=df.index.max()
print(max_val)
min_val=df.index.min()
print(min_val)
span1=len(pd.date_range(min_val,max_val,freq='M'))+1
print(span1)


In [ ]:
# create a df with all the months included within the lowest and highest datastamp
dti = pd.date_range(min_val, periods=span1, freq="MS")
dti=dti.to_series().to_frame()
df=pd.concat([dti,df], axis=1)
df=df.drop([0],axis=1)
df.tail()

### 7. fill missing YTD values

In [ ]:
# use backfill to fill all the YTD values missing in the series 
'''
T: Add data where the values are Nan 
A: Create a copy of the existing dataframe where all NaN rows will be filled with calculated values based on the 
YTD position in December as a proportion based on the missing month and concatenate the complete Values column
to the existing df for comparison. 
'''
# check for null values
df.isna().sum()

In [ ]:
ytd_values=df.copy()
ytd_values=ytd_values[ytd_values.index.month==12]
ytd_values.tail(4)

In [ ]:
Nan_replacement = pd.date_range(min_val, periods=span1, freq="MS")

In [ ]:
Nan_replacement=Nan_replacement.to_series().to_frame()
Nan_replacement

In [ ]:
Nan_replacement=pd.concat([Nan_replacement,ytd_values], axis=1)
Nan_replacement

In [ ]:
Nan_replacement=Nan_replacement.drop([0],axis=1)
Nan_replacement

In [ ]:
Nan_replacement['Month']=Nan_replacement.index.month
Nan_replacement['Year']=Nan_replacement.index.year
Nan_replacement['Original_YTD_values']=Nan_replacement['Original_values'].fillna(method='bfill')
Nan_replacement['NAN_replacement']=Nan_replacement['Original_YTD_values']/12*Nan_replacement['Month']
Nan_replacement.tail(3)

In [ ]:
#test

#df.loc['2021-09-01','Original_values']=np.nan
df=pd.concat([df,Nan_replacement['NAN_replacement']], axis=1)

In [ ]:
#Add 1/2 value based on 'Filled_values' column, into Final_YTD column to estimate the YTD values missing in the oringal data
def final_ytd(df):
    """A function to replace NAN values with a proportion of the final values"""
    if pd.isna(df['Original_values']):#|df['Original_values']==0:
        return df['NAN_replacement']
    else:
        return df['Original_values']
        #return 1
    
df['Final_Ytd']=df.apply(final_ytd,axis=1)

In [ ]:
df.tail(5)

### 8. identify monthly movements

In [ ]:
# use YTD movements to find monthly movements
'''

T: Values need to be converted into monthly movements so that it can be used by the model
A: Convert values from YTD to monthly movements. This is done by subtracting the value of the current observation 
against the value of the previous observation. 
This approach nevertheless causes a problem when we are removing December data from January data. To overcome this problem, 
the existing monthly movement, for January, is replaced by its YTD observation (either original or calculated)
'''

df_ytd=df.drop(['Original_values','NAN_replacement'],axis=1)
df_diff=df_ytd.diff()
df_diff=df_diff.rename(columns={'Final_Ytd':'Monthly_variance'})
df_final=pd.concat([df,df_diff], axis=1)
df_final['Month']=df_final.index.month

# set the monthly value of Jan equal to the YTD value of Jan

def final_value(df):
    """A function to calculate monthly values"""
    if df['Month']==1 :
        return df['Final_Ytd']
    else:
        return df['Monthly_variance']
df_final['Monthly_values']=df_final.apply(final_value,axis=1)
print('df len: ',len(df_final))

In [ ]:
df_final.head(5)

In [ ]:
df_final.tail(10)

In [ ]:
last_monthly_value =df_final.iloc[len(df_final)-1,:]['Monthly_values']
last_monthly_value

In [ ]:
df_final['Monthly_values'].describe()

### 9. Data Analysis of the whole TimeSeries

#### a. Visual Analysis - Original, YTD and Monthly Mov

In [ ]:
# visualize the data

'''
T:Ensure the timeseries is stationary
A: Use all available tools (visual, statistical) to validate the time series is static
'''
plt.figure(figsize=(12, 8))
plt.plot(df_final['Original_values'], '-o', label="YTD Actuals")
plt.plot(df_final['Final_Ytd'], '--*', label="YTD revised")
plt.plot(df_final['Monthly_values'], '-o', label="Monthly Movements")
plt.title("YTD Actuals, YTD revised, Monthly Movements £")
plt.xlabel('Months')
plt.ylabel("£")
plt.legend(fontsize=12)
plt.show()


The monthly movements seem fairly stationary. More analysis is done further down

In [ ]:
# Remove all features which are not needed for the model, to leave only monthly movements
ts=df_final.copy()
ts=ts.drop(['Original_values','NAN_replacement','Final_Ytd','Monthly_variance','Month'],axis=1)
ts=ts.dropna(subset=['Monthly_values'])
ts=ts.rename(columns={'Monthly_values':'Values'})
ts.tail()

In [ ]:
moving_avg = ts.rolling(12).mean()

In [ ]:
ts.describe()

#### b.  Visual analysis - monthly movements only

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(ts['Values'], '-o', label="Monthly Movements")
plt.plot(moving_avg, color='red')
plt.title(" Monthly Movements £")
plt.xlabel('Months')
plt.ylabel("£")
plt.legend(fontsize=12)
plt.show()

#### c. Descriptive Statistics

In [ ]:
plt.hist(ts)
plt.title("Ts values distribution")
plt.xlabel('£')
plt.ylabel("Frequency")
plt.legend(fontsize=12)

In [ ]:
# identify values' distribution: min, max, mean and std, and outlier.

ts.describe()

In [ ]:
#  Test Normality
'''
Use the Shapiro test to assess normality.
H0: The Shapiro-Wilk test tests the null hypothesis that the data was drawn from a normal distribution
H1: The test rejects the hypothesis of normality when the p-value is less than or equal to 0.05
'''

df_shapiro = shapiro(ts)
print('The c test p value is', df_shapiro.pvalue)
if df_shapiro.pvalue > 0.05:
    print('As the p-value from the Shapiro-test (', df_shapiro.pvalue,')>0.05, the residuals are drawn from a normal distribution')
else:
    print('As the p-value from the Shapiro-test (', df_shapiro.pvalue,')<=0.05, the residuals are NOT drawn from a normal distribution. This is not a problem as Normally distributed data is NOT required for Time Series analysis')
  


### 10. Address outliers

In [ ]:
# calculate summary statistics (mean and std deviation) and outliers' thresholds (beyond 3 std deviations)
data_mean, data_std = mean(ts['Values']), std(ts['Values'])
# identify outliers
cut_off = data_std * 3
lower, upper = data_mean - cut_off, data_mean + cut_off
print('Mean:  ', data_mean, 'std:   ',data_std, 'Lower: ', lower, ' Upper: ', upper)

Another way to retrieve statistical infornation

ts.describe().loc['mean']['Values']

In [ ]:
# identify outliers and replace them with the Values column mean; if nothing is returned, this means there are no outliers
def outlier(df):
    """A function to calculate outliers"""
    if df['Values']>upper  or df['Values']<lower:
        return 1
    else:
        return 0
    
ts['outlier']=ts.apply(outlier,axis=1)
ts[ts['outlier']==1]


In [ ]:
# replace outliers value with the df mean
df_mean=ts['Values'].mean()
df_mean

def update_df(df):
    """A function to relace outliers with mean values"""
    if df['outlier']==1:
        return df_mean   
    else:
        return df['Values']
ts['Values']=ts.apply(update_df,axis=1)

ts=ts.drop(columns=['outlier'])
#ts[:12]

<a id='Stationarity'></a>

# Stationarity

### 11 . Serie Stationarity

#### a. Decomposition

In [ ]:
# Identify trend, seasonal and residual components of the series

'''
Decompose the series into Trend, Seasonality and Residuals
'''

decomposition = seasonal_decompose(ts)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Plot the decomposition

plt.figure(figsize=(18, 6), dpi=80)
plt.subplot(411)
plt.plot(ts, label = 'Monthly Movements £')
plt.legend(loc = 'best')
plt.subplot(412)
plt.plot(trend, label = 'Trend')
plt.legend(loc = 'best')
plt.subplot(413)
plt.plot(seasonal, label = 'Seasonality')
plt.legend(loc = 'best')
plt.subplot(414)
plt.plot(residual, label = 'Residuals')
plt.legend(loc = 'best')

Result: the series is not stationary as the trend part of the plot shows a decreasing trend which needs to be addressed.

#### b. Differencing to make the series stationary 

In [ ]:
ts_before_dif=ts.copy()

In [ ]:
# differencing the series to remove the trend and make it stationery
ts=ts.diff(1).dropna()

In [ ]:
#ts[-14:]

In [ ]:
# Define the trend, seasonal and residual components again on the differentiated serie
'''
Decompose the differentiated series into Trend, Seasonality and Residuals
'''

decomposition = seasonal_decompose(ts)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid


plt.figure(figsize=(18, 6), dpi=80)

# Plot the results
plt.subplot(411)
plt.plot(ts, label = 'Actuals Monthly Movemements £')
plt.legend(loc = 'best')
plt.subplot(412)
plt.plot(trend, label = 'Trend')
plt.legend(loc = 'best')
plt.subplot(413)
plt.plot(seasonal, label = 'Seasonality')
plt.legend(loc = 'best')
plt.subplot(414)
plt.plot(residual, label = 'Residuals')
plt.legend(loc = 'best')
plt.tight_layout()

The series now looks stationary as the trend does not show any increment or decrease over time..

#### c. Testing stationarity

In [ ]:
# Use ADF to test for stationarity

'''
If result at 5%  > ADF result  AND P-value <= (0.05),'the series is stationary')
'''


result = adfuller(ts, autolag='AIC')
print('ADF Statistic: %f' % result[0])
print('p-value: %.9f' % result[1])
print(f'n_lags:{result[2]}')
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
    
if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
    #print("\u001b[32mTime Series is Stationary\u001b[0m")
    print ('If result at 5% (',result[4]['5%'], ') > ADF result (', result[0], ') AND P-value:(%.9f '%result[1],') <= (0.05)','the series is stationary')
else:
    print("\x1b[31mTime Series is Non-stationary\x1b[0m")
    

### d. Copies of df for alternative modelling approaches

In [ ]:
ts_copy=ts.copy()

In [ ]:
# due to differencing, the first row of the TS is dropped. 
ts.shape

In [ ]:
#max_val

In [ ]:
# future forecasting dates to use by models:

date_after_month = datetime.today()+ relativedelta(months=1)
future=pd.date_range(max_val+ relativedelta(months=0),max_val+ relativedelta(months=12),freq='MS')
future

### e.   Identify train and validation set

In [ ]:
# the time series is now split into 2 parts:
'''
train set (also called “in-sample data”): the size of this set should be between 70% to 80% of the total dataset
test set (also called “hold-out set” or  “out-of-sample data” because these data are “held out” of the data used for fitting)
'''

train_test_split_per=0.7
l_train=int(len(ts)*train_test_split_per) # find out the data point where the train set stops
l_train=48 # use this if you want a constant training set size

l_test= len(ts)-l_train # find out the data point where the test set starts

#print(len(ts),l_train, l_test)
train =  ts[:-l_test] # identify the train set
test =  ts[l_train:] # identify the test set

print('Train: ',train.shape, 'Test: ', test.shape, 'Ts: ', ts.shape)

<a id='Parameters'></a>

In [ ]:
train

# Arima's parameters

### 21. ARIMA model family approach (overview of the model over the whole time series) 

#### a. PACF (to find value of p for AR() model)

In [ ]:
# Plot the PACF function to identify auto correlation of data with previous values
N, M = 12, 6
fig, ax = plt.subplots(figsize=(N, M))
plot_pacf(train['Values'], lags = 10, title='PACF plot', ax=ax, method='ywmle')
plt.xlabel('Lags')
plt.ylabel("PACF")
plt.show()

#### b. ACF (to find value of q for MA() model)

In [ ]:
# Plot the ACF function to identify auto correlation of data with previous values
N, M = 12, 6
fig, ax = plt.subplots(figsize=(N, M))
plot_acf(train['Values'], lags = 10, title='ACF plot', ax=ax)
#plt.xlabel('Lags')
#plt.ylabel("ACF")
plt.show()

#### c. Find the values of p for the ARIMA models

In [ ]:
'''
Identify the best value for p which tells how many timepoints (=time lag) 
should be taken into consideration for the regression.
'''

# identify optimal lag
model = ar_select_order(train, maxlag=10, ic="aic")
optlag = max(model.ar_lags)
print('Optimal p (with AIC Information Criterion) =', optlag)


p=9 is confirmed to be the best value to use with AR model, according to ar_select_order method.

In [ ]:
# determine the  best values of p (from the list made of PACT and ar_slect_order's findings)
plt.figure(figsize=(12, 12))

ar_orders=[1,2,5,8,9] # based on AR_select_order and the PACF graphs
fitted_model_dict={}
aic_results=[]
aic_p_number=[]

for idx, ar_order in enumerate(ar_orders):
    ar_model = ARIMA(train, order=(ar_order,0,0))
    ar_model_fit = ar_model.fit()
    fitted_model_dict[ar_order]=ar_model_fit

    
    plt.subplot(5,1,idx+1)
    plt.plot(train)
    plt.plot(ar_model_fit.fittedvalues)
    plt.title('AR%s fit' %ar_order,fontsize=16)
    aic_results.append(fitted_model_dict[ar_order].aic)
    aic_p_number.append(ar_order)
    print('AIC for AR(%s): %s'%(ar_order, fitted_model_dict[ar_order].aic),
          ' - BIC for AR(%s): %s'%(ar_order, fitted_model_dict[ar_order].bic))
plt.tight_layout()

In [ ]:
#print(aic_p_number, aic_results)

In [ ]:
aic_result=pd.DataFrame([aic_p_number, aic_results],index=['aic_p_number','aic_results']).T
aic_result

In [ ]:
best_aic_result=aic_result[aic_result['aic_results']==aic_result['aic_results'].min()]
print(best_aic_result)
best_aic_result['aic_p_number']

<a id='Arima'></a>

# ARIMA's Models

### 22. Forecasting  AR, ARMA, SARIMAX Models 


### Model no 1 - AR model

In [ ]:
# fit an AR model on test data data 
'''
At present the attention will be on the ARIMA (AutoRegressive (AR) Integrated (I) MovingAverage(MA)) models' family:

AR (autoregressive) model: where regression is used on a given number of timepoints (p), 
which capture the behaviour of the series, to predict the next values, as the the assumption 
is that past values can be used to predict the next values
'''

# identify the best parameter for p
mod_ar = ar_select_order(train, maxlag=12, ic="aic")
optlag = max(mod_ar.ar_lags)
#optlag=1
print('Optimal lag p : ',optlag)

# instantiate the AR model
ar = tsa.AutoReg(train, lags=optlag)

# fit the AR model on train data
arfit = ar.fit()

 # get the lenght of test data, identify lenght of the test data
# make predictions on unseen data:
prediction_ar = arfit.predict(end=ts.index[-1])[-len(test):]


# Print MAE 
MAE_AR=mean_absolute_error(test, prediction_ar)
print("AR's MAE on test data = {0:.3f}".format(MAE_AR))

# Print MAPE 
MAPE_AR=mean_absolute_percentage_error(test, prediction_ar)
print("AR's MAPE on test data = {0:.3f}".format(MAPE_AR))

# Print RMSE
RMSE_AR=mean_squared_error(test, prediction_ar, squared=False)**0.5
print("AR's RMSE on test data = {0:.3f}".format(RMSE_AR))

# Print MASE
MASE_AR=MeanAbsoluteScaledError()
MASE_AR=MASE_AR(test, prediction_ar, y_train=train)
print("AR's MASE on test data = {0:.3f}".format(MASE_AR))

# plot the forecasts against the true values
plt.figure(figsize=(16, 8))
plt.plot(ts, '-o', label="Actuals Monthly Movemements")
plt.plot(prediction_ar, '--o', label='Prediction')
plt.title("AR Model")
plt.xlabel('Months')
plt.ylabel("Monthly Values £")
plt.legend(fontsize=12)

In [ ]:
# predict into the future
future_prediction_ar = arfit.predict(end=future[-1])[-len(future):]
future_prediction_ar

# plot the forecasts against the future
plt.figure(figsize=(16, 8))
plt.plot(ts, '-o', label="Actuals Monthly Movemements")
plt.plot(prediction_ar, '--o', label='Prediction')
plt.plot(future_prediction_ar, '--x', label='Future Predictions')
plt.title("AR Model")
plt.xlabel('Months')
plt.ylabel("Monthly Values £")
plt.legend(fontsize=12)

In [ ]:
# zoomed future  predictions
plt.figure(figsize=(16,6))
plt.plot(test, '-o', label="test data")
plt.plot(prediction_ar, '--o', label='prediction')
plt.plot(future_prediction_ar, '--x', label='prediction')
plt.title("AR Model")
plt.xlabel('Months')
plt.ylabel("Monthly Values £")
plt.legend(fontsize=12)
plt.show()

### Model no 2 - ARIMA model (taken from ARIMA class where d=0)

In [ ]:
# fit an ARIMA model on test data data 

'''
MA model: where the moving average (MA) of lagged past errors (q) is added to the AR model to account for different behaviours
ARIMA model: which is able to process non-stationary data by including the integrated parameter (d), which counts the numbers of differentiations until stationarity is reached, to underlying ARMA model.

  ARIMA model therefore requires not only stationarity, but also the identification of the value of 3 parameters called p, d and q.

  - p (relevant for AR)is the number of autoregressive terms , 
  - d (relevant for I) is the number of nonseasonal differences needed for stationarity, and 
  - q (relevant for MA) is the number of lagged forecast errors in the prediction equation.
'''

# The timeseries has already be differentiated so d=0 as no additional differentiations are neeeded.

# identify the best parameter for p and q 
param_choice = tsa.arma_order_select_ic(train, max_ar=12, max_ma=4, ic='aic', trend='c')
print('Best of option for ARMA(p,q) =',param_choice['aic_min_order'])

par_p=param_choice['aic_min_order'][0]
par_q=param_choice['aic_min_order'][1]

# as data is stationary, d=0 as the model does not need a further differentiation
arima = sm.tsa.ARIMA(train, order=(par_p,0, par_q))
arima_fit = arima.fit()

# we make predictions on unseen data: 
prediction_ari = arima_fit.predict(end=ts.index[-1])[-len(test):]
#print(prediction_ari)

# Print MAE 
MAE_ARIMA = mean_absolute_error(test, prediction_ari)
print("ARIMA's MAE = {0:.3f}".format(MAE_ARIMA))

# Print MAPE 
MAPE_ARIMA = mean_absolute_percentage_error(test, prediction_ari)
print("ARIMA's MAPE = {0:.3f}".format(MAPE_ARIMA))

# Print RMSE
RMSE_ARIMA=mean_squared_error(test, prediction_ari, squared=False)**0.5
print("ARIMA's RMSE on test data = {0:.3f}".format(RMSE_ARIMA))

# Print MASE
MASE_ARIMA=MeanAbsoluteScaledError()
MASE_ARIMA=MASE_ARIMA(test, prediction_ari, y_train=train)
print("ARIMA's MASE on test data = {0:.3f}".format(MASE_ARIMA))

# plot predictions v true data
plt.figure(figsize=(16, 8))
plt.plot(ts, '-o', label="Actuals Monthly Movemements")
plt.plot(prediction_ari, '--o', label='Prediction')
plt.title("ARIMA Model")
plt.xlabel('Months')
plt.ylabel("Monthly Values £")
plt.legend(fontsize=12)

In [ ]:
# predict into the future
future_prediction_arima = arima_fit.predict(end=future[-1])[-len(future):]
future_prediction_arima

# plot the forecasts against the future
plt.figure(figsize=(16, 8))
plt.plot(ts, '-o', label="Actuals Monthly Movemements")
plt.plot(prediction_ari, '--o', label='Prediction')
plt.plot(future_prediction_arima, '--x', label='future predictions')
plt.title("ARIMA Model")
plt.xlabel('Months')
plt.ylabel("Monthly Values £")
plt.legend(fontsize=12)

In [ ]:
# zoomed future ARIMA predictions

plt.figure(figsize=(16,6))
plt.plot(test, '-o', label="test data")
plt.plot(prediction_ari, '--o', label='prediction')
plt.plot(future_prediction_arima, '--x', label='prediction')
plt.title("ARIMA Model")
plt.xlabel('Months')
plt.ylabel("Monthly Values £")
plt.legend(fontsize=12)
plt.show()

### Model no 3 - Sarimax Model

In [ ]:
# initialize Sarimax model with the best hyperparameter identified in the search at the end of the Jupiter Book

'''
On top of the ARIMA's model, the SARIMAX (Seasonal Auto-Regressive Integrated Moving Average with 
eXogenous factors) model accounts also for seasonality. The notation for a SARIMAX model is specified as:SARIMA(p,d,q)(P,D,Q,m)

    Configuring a SARIMA requires therefore selecting hyperparameters for both the trend (p,d,q), 
    mentioned above, and seasonal elements of the series (P,D,Q,m) which mean:.

    - P: Seasonal autoregressive order.
    - D: Seasonal difference order.
    - Q: Seasonal moving average order.
    - m: The number of time steps for a single seasonal period.
'''
# mod_sar = SARIMAX(train, trend='c', # use this if train set size = 70%
#               order=(3,0,1), 
#               seasonal_order=(2,0,2,12),
#               enforce_stationarity=False, 
#               enforce_invertibility=False)

mod_sar = SARIMAX(train, trend='c', # use this when train set size = 48 
              order=(1,0,3), 
              seasonal_order=(3,0,3,12),
              enforce_stationarity=False, 
              enforce_invertibility=False)
#fit model
sarima_fit= mod_sar.fit()

# run prediction
prediction_sar = sarima_fit.predict(end=ts.index[-1])[-len(test):]

# calculate MAE
MAE_SARIMAX=mean_absolute_error(test, prediction_sar)
print("SARIMAX' s MAE = {0:.3f}".format(MAE_SARIMAX))

# calculate MAPE
MAPE_SARIMAX=mean_absolute_percentage_error(test, prediction_sar)
print("SARIMAX' s MAPE = {0:.3f}".format(MAPE_SARIMAX))

# Print RMSE
RMSE_SARIMAX=mean_squared_error(test, prediction_sar, squared=False)**0.5
print("SARIMAX's RMSE on test data = {0:.3f}".format(RMSE_SARIMAX))

# Print MASE
MASE_SARIMAX=MeanAbsoluteScaledError()
MASE_SARIMAX=MASE_SARIMAX(test, prediction_sar, y_train=train)
print("SARIMAX's MASE on test data = {0:.3f}".format(MASE_SARIMAX))

# plot predictions v true data
plt.figure(figsize=(16, 8))
plt.plot(ts, '-o', label='true')
plt.plot(prediction_sar, '-o', label='model')
plt.title("SARIMAX Model")
plt.xlabel('Months')
plt.ylabel("Monthly Values £")
plt.legend();
plt.show()
#print(test)
#print('-'*20)
#print(prediction_x)

In [ ]:
# predict into the future
future_prediction_sar = sarima_fit.predict(end=future[-1])[-len(future):]
future_prediction_sar

# plot the forecasts against the future
plt.figure(figsize=(16, 8))
plt.plot(ts, '-o', label="Actuals Monthly Movemements")
plt.plot(prediction_sar, '--o', label='Prediction')
plt.plot(future_prediction_sar, '--x', label='Future Predictions')
plt.title("SARIMAX Model")
plt.xlabel('Months')
plt.ylabel("Monthly Values £")
plt.legend(fontsize=12)

In [ ]:
# zoomed FUTURE predictions
plt.figure(figsize=(18, 6))
plt.plot(test, '-o', label="test data")
plt.plot(prediction_sar, '--o', label='prediction')
plt.plot(future_prediction_sar, '--x', label='prediction')
plt.title("SARIMAX Model")
plt.xlabel('Months')
plt.ylabel("Monthly Values £")
plt.legend(fontsize=12)
plt.show()

In [ ]:
# identify residuals (=  difference between the true values (starting at the position of the optimal lag) and the predictions)
plt.figure(figsize=(12, 8))
fit_residuals = test['Values'] - prediction_sar 
#res_moving_avg = fit_residuals.rolling(12).mean()

# Plot the residuals
plt.plot(fit_residuals, '-o', label="Residuals")
#plt.plot(res_moving_avg, color='red')
plt.title('Residuals', fontsize=12)
plt.legend(fontsize=12);
plt.xlabel('Months')
plt.ylabel("Monthly Values £")
plt.show()

In [ ]:
# the histogram shows values are around the mean of zero with a std deviation between -400 and 400 with a few outlier 
plt.hist(fit_residuals, bins=8)
plt.title("Residuals Distribution ")
plt.xlabel('£')
plt.ylabel("Frequency")
plt.show()

In [ ]:
fit_residuals.describe()

In [ ]:
print ('Mean of residuals:%.9f '%fit_residuals.describe()[1])

In [ ]:
N, M = 12, 6
fig, ax = plt.subplots(figsize=(N, M))
plot_acf(fit_residuals, lags = 20, title='ACF plot', ax=ax)
plt.xlabel('Lags')
plt.ylabel("ACF")
plt.show()

In [ ]:
# Use the Shapiro test to assess normality.
# H0: The Shapiro-Wilk test tests the null hypothesis that the data was drawn from a normal distribution
#H1: The test rejects the hypothesis of normality when the p-value is less than or equal to 0.05

test_shapiro = shapiro(fit_residuals)
print('The Shapiro-Wilk test p value is', test_shapiro.pvalue)
if test_shapiro.pvalue > 0.05:
    print('As the p-value from the Shapiro-test (', test_shapiro.pvalue,')>0.05, the residuals are drawn from a normal distribution')
else:
    print('As the p-value from the Shapiro-test (', test_shapiro.pvalue,')<=0.05, the residuals are NOT drawn from a normal distribution')
  


<a id='Prophet'></a>

# ------------------------------------ Additional models    ------------------------------------

### 23. Forecasting  Prophet Model

In [ ]:
pr=ts_before_dif.copy()      #ts_copy.copy() # copy for prophet modelling

In [ ]:
#rename columns to comply to Prophet's requirements
'''
The Prophet library is library designed for making forecasts for univariate time series datasets 
where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. 

Therefore, it is not necessary to differentiate the series

This is a additive regression model which considers 3 elements:

𝑦(𝑡)=𝑔(𝑡)+𝑠(𝑡)+ℎ(𝑡)+𝜖𝑡
'''
pr.reset_index(level=0, inplace=True)
pr.rename(columns={'index':'ds', 'Values':'y'},inplace=True)
pr.tail()

In [ ]:
#df_future_dates=future.to_frame().reset_index().drop(0, axis=1).rename({'index':'ds'})

In [ ]:
train_pr =  pr[:-l_test] # identify the train set
test_pr =  pr[l_train:]

In [ ]:
test_pr.shape

In [ ]:
# instantiate model and fit the data
forecast_model = Prophet()   #Prophet(growth='linear', daily_seasonality=False)
forecast_model_pr=forecast_model.fit(train_pr)
pr_pred=forecast_model_pr.predict(test_pr)

# Calculate and print MAE
MAE_PROPHET=mean_absolute_error(test_pr.loc[:,'y'],pr_pred[:ts.shape[0]]['trend'])
print("Prophet's MAE = {0:.3f}".format(MAE_PROPHET))

# Calculate and print MAPE
MAPE_PROPHET=mean_absolute_percentage_error(test_pr.loc[:,'y'],pr_pred[:ts.shape[0]]['trend'])
print("Prophet's MAPE = {0:.3f}".format(MAPE_PROPHET))

# Calculate and print RMSE
RMSE_PROPHET=mean_squared_error(test_pr.loc[:,'y'],pr_pred[:ts.shape[0]]['trend'])**0.5
print("Prophet's RMSE = {0:.3f}".format(RMSE_PROPHET))

# Calculate and print MASE
MASE_PROPHET=MeanAbsoluteScaledError()
MASE_PROPHET=MASE_PROPHET(test_pr.loc[:,'y'], pr_pred[:ts.shape[0]]['trend'], y_train=train_pr.loc[:,'y'])
print("Prophet's MASE = {0:.3f}".format(MASE_PROPHET))

#pr_pred.shape

In [ ]:
# work out the dates the model will use
df_dates = forecast_model_pr.make_future_dataframe(periods=12, 
                                                freq='M',
                                                include_history=True)


In [ ]:
# calculate predictions (they are stored in a dataframe with yhat, trend, additive, yearly, multiplicative values )
model_predictions_pr = forecast_model_pr.predict(df_dates)
#model_predictions_future = forecast_model.predict(df_future_dates)

#retrieve the trend data from the dataframe
pr_pred=pd.DataFrame(model_predictions_pr['trend'])
#pr_pred

In [ ]:
# Plot timeseries and forecasts
plt.figure(figsize=(12, 8))
#plot_pred = forecast_model_pr.plot(model_predictions_pr)
fig = forecast_model_pr.plot(model_predictions_pr)


changepoint_plot = add_changepoints_to_plot(fig.gca(), forecast_model_pr, model_predictions_pr)

plt.title("Prophet Model")
plt.xlabel('Months')
plt.ylabel("Monthly Values £")
plt.show();

In [ ]:
plot_components = forecast_model_pr.plot_components(model_predictions_pr, uncertainty=False)

<a id='NN'></a>


### 24.  Neural Network

In [ ]:
nn = ts_copy.copy() # copy for neural network modelling

In [ ]:
# make a copy ts timeseries and keep only the values column
'''
eneral characteristics of LSTM's RNN:

1 - In the RNN context, dates are not important 
as the timing of each observation is constant (monthly) 

2 - LSTMs are sensitive to the scale of the input data, and therefore the data needs to be 
normalized (rescaled in a 0 to 1 range)

3 - The LSTM network expects the input data (X) to be provided with a specific 
array structure in the form of: [samples, time steps, features].
'''

nn_df=nn
#nn_df.info()
dataset = nn_df.values

# ensure the new dataset has float32 format 
# note: dataset shape is the same as ts (= no yrs analysed x 12 (no months) - 1 (for differencing)) with 1 column only (values)
dataset = dataset.astype('float32')
print(dataset.shape)
print( '6 years worth of monthly data "2016 to 2021" (72 data points) reduced by one due to differencing (now 71)')

In [ ]:
# function to transform the df into a matrix
def create_dataset(dataset, look_back=1):
    """function to transform the df into a matrix"""
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1): 
        a = dataset[i:(i+look_back), 0] # create matrix  (dataX) with moving starting position and constant width 
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])# create array (dataY) having the following datapoint (on the right) of dataX
    return np.array(dataX), np.array(dataY)
# fix random seed for reproducibility
np.random.seed(11)

In [ ]:
# normalize the original dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

# split the original dataset into train and test sets
train_size = l_train #int(len(dataset) * train_test_split_per)
test_size = l_test #len(dataset) - train_size
train_NN = dataset[0:train_size,:]
test_NN =  dataset[train_size:len(dataset),:]
print('Train size: ', train_size, 'Test size: ', test_size, 'Ts size: ', len(ts))

# reshape into X=t and Y=t+1 --> shape will be (m x n) 
look_back = 3
trainX, trainY = create_dataset(train_NN, look_back)
testX, testY = create_dataset(test_NN, look_back)
print('Reshaped Train size: ', len(trainX), 'Reshaped Test size: ', len(testX))

In [ ]:
# reshape input to be [samples, time steps, features] --> shape will be (m x n x 1)
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))
trainX.shape

In [ ]:
testY

In [ ]:
# create and fit the LSTM network

# define batch size
batch_size = 1

# initiate sequential model
model = Sequential()

# add layers, dense layer and optimizer to the model
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True, return_sequences=True))
model.add(LSTM(4, batch_input_shape=(batch_size, look_back, 1), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# fit data into the model to optimize the output
for i in range(100):
    model.fit(trainX, trainY, epochs=1, batch_size=batch_size, verbose=2, shuffle=False)
    model.reset_states()

In [ ]:
# make predictions using optimized model
trainPredict = model.predict(trainX, batch_size=batch_size)
model.reset_states()
testPredict = model.predict(testX, batch_size=batch_size)
testPredict

In [ ]:
# invert predictions from a scaler value to its original value
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])

testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
print(testY.shape)

In [ ]:
testY

In [ ]:
# invert train set from a scaler value to its original value
# trainX = scaler.inverse_transform(trainX[0])
# trainX

In [ ]:
# Calculate MAE
MAE_NN=mean_absolute_error(testY[0], testPredict[:,0])
print("NN's MAE = {0:.3f}".format(MAE_NN))

# Calculate MAPE
MAPE_NN=mean_absolute_percentage_error(testY[0], testPredict[:,0])
print("NN's MAPE = {0:.3f}".format(MAPE_NN))

# Calculate RMSE
RMSE_NN=mean_squared_error(testY[0], testPredict[:,0])**0.5
print("NN's RMSE = {0:.3f}".format(RMSE_NN))


# Calculate MASE
MASE_NN=MeanAbsoluteScaledError()
MASE_NN=MASE_NN(testY[0],testPredict[:,0], y_train=trainX[0])
print("NN's MASE = {0:.3f}".format(MASE_NN))



In [ ]:
# plot the graph
plt.figure(figsize=(12,8))
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.title("Neural Network Model - Train and Test  data")
plt.xlabel('Months (in progressive order) )')
plt.ylabel("Monthly Values £")
#plt.legend(fontsize=12)
plt.show()

<a id='XG'></a>

 ### 25. Forecasting  XGBoost

In [ ]:
xg = ts_copy.copy() # copy for XGBoost modelling
#xg 

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
'''
XGBoost (Extreme Gradient Boosting) is an implementation of gradient boosting used for classification and regression problems.
It is an ensemble of decision trees algorithm where new trees fix errors of those trees that are already part of the model. 
Trees are added until no further improvements can be made to the model.

XGBoost can also be used for time series forecasting. To do so, the time series dataset needs to be 
transformed into a supervised learning problem first. This can be done by using the slide window representation 
where existing values are used as input variables and the immediate next time step as the output variable.

To evaluate the model, the walk-forward validation needs to be used, where the time based nature of the data is taken 
into consideration. For this reason, k-fold cross validation cannot be used.
'''
# transform a time series dataset into a supervised learning dataset 
# by aligning existing values (used as input) to its next predefined no of values (used as output)

# this is the preparation sample data to train XGboost model (each row correspond to 1 sample) 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True): 
    """A function to set up the ts as a df with n_in width (1 by default) """
    df = pd.DataFrame(data)
    cols = list()
    # input sequence (t-n, ... t-1)
     #col1 = copy a fixed amount of datapoint (equal to len(df), ie 71 datapoints) from the beginning of the df, n_in times
        # these copies are stacked in 1 column, and each copy (of this fixed length df) begins with a decreasing no of nan values 
        # these nan values are of n_in amount, to start with, and then they go down to zero at each copy of these datapoints.
        # the rows with nans are then dropped
    for i in range(n_in, 0, -1): 
        cols.append(df.shift(i))
    #print('col 1st------------------------------------>',cols)
        
    # forecast sequence (t, t+1, ... t+n) (prepare 1 additional column only at the end of the n_in number of columns)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    #print('col 2nd----------------------------------->',cols)
        
    # put it all together --> the new dataframe will have n_in+1 columns
    agg = pd.concat(cols, axis=1)
    # drop rows with NaN values (those at the beginning of the dataframe; this should be n_in number of rows)
    if dropnan:
        agg.dropna(inplace=True)
    #print(agg)
    #print ('agg.values: --------------------------->', agg.values)
    return agg.values

In [ ]:
# split the dataset into train/test sets
def train_test_split(data, n_test):
    """A function to split data in train an test"""
    #print('data1 ----------------------->.',  data[:-n_test, :])
    #print('data2 -------------------------->',data[-n_test:, :])
    return data[:-n_test, :], data[-n_test:, :]

# fit an xgboost model and make a one step prediction
def xgboost_forecast(train, testX):
    """A function to fit XGBoost"""
    # transform list into array
    train = np.asarray(train)
    # split into input and output columns
    trainX = train[:, :-1]
    #print('trainX--------------->',trainX)
    trainy = train[:, -1]
    #print('trainy--------------->',trainy)
    # fit model
    model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
    model.fit(trainX, trainy)
    # make a one-step prediction
    yhat = model.predict(np.asarray([testX]))
    #print('yhat-------------------------------->', yhat)
    return yhat[0]
 
# walk-forward validation for univariate data

def walk_forward_validation(data, n_test):
    """A function to validate the data with walk_forward"""
    predictions = list()
    # split dataset
    train, test = train_test_split(data, n_test)
    #print('train---------------------------->',train)
    print()
    #print('test--------------------------->',test)
    # seed history with training dataset
    history = [x for x in train]
    #print('history: ---------------------------------------------->',history)
    # step over each time-step in the test set
    for i in range(len(test)):
        # split test row into input and output columns
        testX, testy = test[i, :-1], test[i, -1]
        #print(testX,testy)
        # fit model on history and make a prediction
        yhat = xgboost_forecast(history, testX)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
        # summarize progress
        print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
    # estimate prediction error
    #print('history-------------------->', history)
    #print('predictions----->',predictions)
    error = mean_absolute_error(test[:, -1], predictions)
    error_percentage = mean_absolute_percentage_error(test[:, -1], predictions)
    error_RMSE= mean_squared_error(test[:, -1], predictions)**0.5
    error_MASE=MeanAbsoluteScaledError()
    error_MASE=error_MASE(test[:, -1],pd.DataFrame(predictions), y_train=data[:-n_test, -1])

    return error, error_percentage, error_RMSE,error_MASE, test[:, -1], predictions
 

In [ ]:
XG_values =xg.values

In [ ]:
# transform the time series data into supervised learning
# keep the existing observation's value (value no 1 from the df) and add the next 6 observations; 
# then move the next observation (no 2) and add the next 6 observations and so on
XG_data = series_to_supervised(XG_values, n_in=6)
#XG_data

In [ ]:
# evaluate model 
MAE_XG, MAPE_XG, RMSE_XG, MASE_XG,y, yhat = walk_forward_validation(XG_data, 23)
#print('mae',MAE_XG, 'mape',MAPE_XG, 'rsme', RSME_XG,'y', y, 'yhat', yhat)
# print(y)'
# print(yhat)
print('MAE: %.3f' % MAE_XG)
print('MAPE: %.3f' % MAPE_XG)
print('RMSE: %.3f' % RMSE_XG)
print('MASE: %.3f' % MASE_XG)

In [ ]:
# plot expected vs predicted
plt.figure(figsize=(12,8))
plt.plot(y, label='Actuals Monthly Movements £')
plt.plot(yhat, label='Predicted')
plt.legend()
plt.title("XGBoost Model - Test data")
plt.xlabel('Last 23 Months of TS')
plt.ylabel("Values £")
plt.show()

Although this XGBoost is performing better the RNN model, the variance is still high. Its mae is 642 against 628 of RNN and 272 of SARIMAX. It has learned the signal much better than RNN but some of the predictions seem 'lagged' from the truth. Adding additional data points should help to achieve better results.

<a id='HW'></a>

### 26. Holt-Winters model

wh_before=ts_before_dif.copy() 
wh_before.to_csv('angry_before.csv')

In [ ]:
HW_df=ts.copy()
HW_df.index.names = ['Month']
HW_df.tail()

## 1. SMA - Simple Moving Averages

In [ ]:
#SMA - Simple Moving Averages
'''
Moving Averages and Single Exponential Smoothing does a poor job of forecasting when there 
is trend and seasonality in the data. Double and Triple exponential smoothing is best suited for this kind of timeseries data.

1 - Winter’s method assumes that the time series has a level, trend and seasonal component. 
2 - Double Exponential smoothing uses a smoothing factor that addresses trend. 
3 - Triple Exponential smoothing uses a smoothing factor that addresses seasonality.
A Holt-Winters model is defined by its three order parameters, alpha, beta, gamma. 
Alpha specifies the coefficient for the level smoothing. 
Beta specifies the coefficient for the trend smoothing. 
Gamma specifies the coefficient for the seasonal smoothing.
'''
#plt.figure(figsize=(12,8))
HW_df.dropna(inplace=True)
HW_df['6-month-SMA'] = HW_df['Values'].rolling(window=6).mean()
HW_df['12-month-SMA'] = HW_df['Values'].rolling(window=12).mean()
HW_df.plot(title='Simple Moving Averages',ylabel=("Monthly Values £"));

In [ ]:
#HW_df.plot(title='Simple Moving Averages',ylabel=("Monthly Values £"));

In [ ]:
HW_df.head(12)

## 2. EWMA - Exponentially Weighted Moving Average

In [ ]:
# EWMA - Exponentially Weighted Moving Average
HW_df['ewma12'] = HW_df['Values'].ewm(span=12,adjust=False).mean()
HW_df[['Values','ewma12']].plot(title='EWMA - Exponential Weighted Moving Avg',ylabel=("Monthly Values £"));
#

In [ ]:
HW_df[['Values','ewma12']].plot(title='EWMA - Exponential Weighted Moving Avg',ylabel=("Monthly Values £"))

In [ ]:
#Comparing SMA to EWMA
HW_df[['Values','ewma12','6-month-SMA','12-month-SMA']].plot();

## 3. SES - Single exponential Smoothing

In [ ]:
# set the span and the smoothing factor alpha
span = 12
alpha = 2/(span+1)

In [ ]:
HW_df.columns

In [ ]:
HW_df.index.freq = 'MS' 
HW_df.head()

In [ ]:
HW_df.index

In [ ]:
HW_df['SES12'] = SimpleExpSmoothing(HW_df['Values']).fit(smoothing_level=alpha,optimized=False).fittedvalues.shift(-1)
HW_df.head()

In [ ]:
HW_df[['Values','ewma12','SES12']].plot(title='EWMA and Holt Winters Single Exponential Smoothing', ylabel='Monthly Values £');

## 4. Holt Exponential Smoothing (HES) - Double Exponential Smoothing

In [ ]:
#Double Exponential Smoothing
HW_df['DES12'] = ExponentialSmoothing(HW_df['Values'],trend='add').fit().fittedvalues.shift(-1)
HW_df[['Values','SES12','DES12']].plot(title='Holt Winters Single & Double Exponential Smoothing', ylabel='Monthly Values £');

In [ ]:
#last 24 months
HW_df[['Values','SES12','DES12']].iloc[:24].plot(title='Holt Winters Single & Double Exponential Smoothing Last 24 months').autoscale(axis='x',tight=True);

In [ ]:
#HW_df['DES12_mul'] = ExponentialSmoothing(HW_df['Values'],trend='add').fit().fittedvalues.shift(-1)

In [ ]:
HW_df[['Values','DES12']].iloc[:24].plot(title='Holt Winters Double Exponential Smoothing(Additive)').autoscale(axis='x',tight=True);

## 5. Holt-Winter (WES) - triple exponential smoothing
HW_df['TESadd12'] = ExponentialSmoothing(HW_df['Values'],trend='add',seasonal='add',seasonal_periods=12).fit().fittedvalues
HW_df.head()

In [ ]:
# tripple exponential smoothing
HW_df['TESadd12'] = ExponentialSmoothing(HW_df['Values'],trend='add',seasonal='add',seasonal_periods=12).fit().fittedvalues
HW_df.head()

In [ ]:
#HW_df['TESmul12'] = ExponentialSmoothing(HW_df['Values'],trend='add',seasonal='add',seasonal_periods=12).fit().fittedvalues
HW_df[['Values','TESadd12']].iloc[:24].plot(title='Holt Winters Triple Exponential Smoothing Last (Additive )').autoscale(axis='x',tight=True);

In [ ]:
HW_df[['Values','TESadd12']].plot(title='Holt Winters Triple Exponential Smoothing Last (Additive )',figsize=(16,8),ylabel='Monthly Values £').autoscale(axis='x',tight=True);

In [ ]:
# Split into train and test set
train_HW_df = HW_df[:l_train] 
test_HW_df = HW_df[l_train:] 

In [ ]:
#test_HW_df

In [ ]:
#Fit the model on the train set 

fitted_model_HW = ExponentialSmoothing(train_HW_df['Values'],
                                    trend='add',seasonal='add',seasonal_periods=12).fit()
predictions_HW = fitted_model_HW.forecast(l_test).rename('HW Test Forecast')
predictions_HW[:10]

In [ ]:
#predictions_HW

In [ ]:
# train_HW_df['Values'].plot(legend=True,label='TRAIN')
# test_HW_df['Values'].plot(legend=True,label='TEST',figsize=(12,8))
# plt.title('Train and Test Data');

In [ ]:
train_HW_df['Values'].plot(legend=True,label='TRAIN')
test_HW_df['Values'].plot(legend=True,label='TEST',figsize=(16,8))
predictions_HW.plot(legend=True,label='PREDICTION')
#plt.xlabel('Values')
plt.ylabel("Monthly Values £")
plt.title('Train, Test and Predicted Test using Holt Winters');

In [ ]:
len(test_HW_df.loc[:,'Values'])

In [ ]:
# Calculate and print MAE
MAE_HW=mean_absolute_error(test_HW_df.loc[:,'Values'],predictions_HW[:ts.shape[0]])

# Calculate and print MAPE
MAPE_HW=mean_absolute_percentage_error(test_HW_df.loc[:,'Values'],predictions_HW[:ts.shape[0]])

# Calculate and print RMSE
RMSE_HW=mean_squared_error(test_HW_df.loc[:,'Values'],predictions_HW[:ts.shape[0]])**0.5

# Calculate and print MASE
MASE_HW=MeanAbsoluteScaledError()
MASE_HW=MASE_HW(test_HW_df.loc[:,'Values'],pd.DataFrame(predictions_HW[:ts.shape[0]]), y_train=test_HW_df.loc[:,'Values'])

print("Holt-Winters' MAE = {0:.3f}".format(MAE_HW))
print("Holt-Winters' MAPE = {0:.3f}".format(MAPE_HW))
print("Holt-Winters' RMSE = {0:.3f}".format(RMSE_HW))
print("Holt-Winters' MASE = {0:.3f}".format(MASE_HW))


In [ ]:
train_HW_df['Values'].plot(legend=True,label='TRAIN')
test_HW_df['Values'].plot(legend=True,label='TEST',figsize=(12,8))
predictions_HW.plot(legend=True,label='PREDICTION',xlim=['2020-01-01','2021-12-01']);

The model seems to have learnt the data fairly well, by judging the charts, but it is again underfitting. The predictions seem to follow the distribution of the true data but their values are quite different. This could be due to the nature of the data which does not really show a strictly exponential nature. The performance metrics for this model, with a MAE of 695, are the worst amongst the performance metrics examined so far. 

In [ ]:
# forecast into the future

In [ ]:
# change mul to add
final_model_HW = ExponentialSmoothing(HW_df['Values'],trend='add',seasonal='add',seasonal_periods=12).fit()

In [ ]:
forecast_predictions_WH = final_model_HW.forecast(steps=36)

In [ ]:
forecast_predictions_WH.plot(legend=True,label='Sales Forecast 2022-2024')
plt.ylabel("Monthly Values £")
plt.xlabel("Month")
plt.title('Sales Forecast 2022-2024');

<a id='Darts'></a>

## 27. Darts

In [ ]:
# split the timeseries into train and test dataset
'''
arts is comparable to other models as it follows the same principles:

1 - It is based and it generates a TimeSeries (containing only datetimes and the value of the observations). 
2 - it uses fit() and predict() methods with is commont to all forecasting modesl 
3 - Darts can be used with the following methods:
'''
darts_train=train.reset_index().rename(columns={'index':'Dates'})
darts_test=test.reset_index().rename(columns={'index':'Dates'})
darts_ts=ts.reset_index().rename(columns={'index':'Dates'})

ds_train = TimeSeries.from_dataframe(darts_train, 'Dates', 'Values')
ds_test = TimeSeries.from_dataframe(darts_test, 'Dates', 'Values')
ds_ts= TimeSeries.from_dataframe(darts_ts, 'Dates', 'Values')
len(ds_test)

In [ ]:
#dir(darts.models)

In [ ]:
from darts.utils.statistics import plot_acf, check_seasonality

plot_acf(ds_train, m=12, alpha=0.05)
for m in range(2, 25):
    is_seasonal, period = check_seasonality(ds_train, m=m, alpha=0.05)
    if is_seasonal:
        print("There is seasonality of order {}.".format(period))

In [ ]:
#train the model and calculate the predition on the test set

model_ds = ES(seasonal_periods=12)#ExponentialSmoothing(seasonal_periods=12)
model_ds.fit(ds_train)
prediction_darts = model_ds.predict(len(ds_test))

In [ ]:
len(ds_train)

In [ ]:
# plot the whole time series and the prediction
plt.figure(figsize=(12,8))
ds_ts.plot(label='actual')
prediction_darts.plot(label='forecast', lw=3)
plt.title('Darts Sales Forecast - With Exponential Smoothing')
plt.xlabel('Months')
plt.ylabel('£')
plt.legend()

df_darts=prediction_darts.pd_dataframe(copy=True)
#df_darts

# calculate MAE
MAE_darts=mean_absolute_error(test, df_darts)

# calculate MAPE
MAPE_darts=mean_absolute_percentage_error(test, df_darts)

# calculate RMSE
RMSE_darts=mean_squared_error(test, df_darts)**0.5

# Calculate MASE
MASE_darts=MeanAbsoluteScaledError()
MASE_darts=MASE_darts(test, df_darts, y_train=train)


print("Darts'(ES) MAE = {0:.3f}".format(MAE_darts))
print("Darts'(ES) MAPE = {0:.3f}".format(MAPE_darts))
print("Darts'(ES) RMSE = {0:.3f}".format(RMSE_darts))
print("Darts'(ES) MASE = {0:.3f}".format(MASE_darts))

In [ ]:
#df_darts

In [ ]:
from darts.models import AutoARIMA

model_aarima = AutoARIMA()
model_aarima.fit(ds_train)
prediction_aarima = model_aarima.predict(len(ds_test))

# plot the whole time series and the prediction
plt.figure(figsize=(12,8))
ds_ts.plot(label='actual')
prediction_aarima.plot(label='forecast', lw=3)
plt.title('Darts Sales Forecast - With Auto-ARIMA')
plt.xlabel('Months')
plt.ylabel('£')
plt.legend()

df_darts_aarima=prediction_aarima.pd_dataframe(copy=True)
#df_darts

# calculate MAE
MAE_darts_AARIMA=mean_absolute_error(test, df_darts_aarima)

# calculate MAPE
MAPE_darts_AARIMA=mean_absolute_percentage_error(test, df_darts_aarima)

# calculate RMSE
RMSE_darts_AARIMA=mean_squared_error(test, df_darts_aarima)**0.5


# # # Calculate MASE
# MASE_darts_aarima=MeanAbsoluteScaledError()
# MASE_darts_aarima=MASE_darts(test, df_darts_aarima, y_train=train)
# print("Dart's MASE = {0:.3f}".format(MASE_darts_aarima))

print("Darts'(AARIMA) MAE = {0:.3f}".format(MAE_darts_AARIMA))
print("Darts'(AARIMA) MAPE = {0:.3f}".format(MAPE_darts_AARIMA))
print("Darts'(AARIMA)RMSE = {0:.3f}".format(RMSE_darts_AARIMA))

In [ ]:
df_darts_aarima

<a id='MAE'></a>

 # ---------------------------   Metrics   ---------------------------------

#### 30. Models' MAE comparison

In [ ]:
print('-'*5,'   MAE   ','-'*5)

#print MAE list showing all models' MAE
print('MAE - AR: {0:.3f} '.format(MAE_AR))
print('MAE - ARMA: {0:.3f} '.format(MAE_ARIMA))
print('MAE - SARIMAX: {0:.3f} '.format(MAE_SARIMAX))
print('MAE - PROPHET: {0:.3f} '.format(MAE_PROPHET))
print('MAE - XGBoost: {0:.3f} '.format(MAE_XG))
print('MAE - Neural Network: {0:.3f} '.format(MAE_NN))
print('MAE - Holt-Winter: {0:.3f} '.format(MAE_HW))
print('MAE - Darts: {0:.3f} '.format(MAE_darts))

MAE_ALL=[MAE_AR,MAE_ARIMA,MAE_SARIMAX,MAE_PROPHET,MAE_XG,MAE_NN,MAE_HW,MAE_darts]

#### 31. Models' MAPE comparison

In [ ]:
print('-'*5,'   MAPE   ','-'*5)

#print MAPE list showing all models' MAPE
print('MAPE - AR: {0:.3f} '.format(MAPE_AR))
print('MAPE - ARMA: {0:.3f} '.format(MAPE_ARIMA))
print('MAPE - SARIMAX: {0:.3f} '.format(MAPE_SARIMAX))
print('MAPE - PROPHET: {0:.3f} '.format(MAPE_PROPHET))
print('MAPE - XGBoost: {0:.3f} '.format(MAPE_XG))
print('MAPE - Neural Network: {0:.3f} '.format(MAPE_NN))
print('MAPE - Holt-Winter: {0:.3f} '.format(MAPE_HW))
print('MAPE - Darts: {0:.3f} '.format(MAPE_darts))

MAPE_ALL=[MAPE_AR,MAPE_ARIMA,MAPE_SARIMAX,MAPE_PROPHET,MAPE_XG,MAPE_NN,MAPE_HW,MAPE_darts]

#### 32. Models' RSME comparison

In [ ]:
print('-'*5,'   RMSE   ','-'*5)

#print RMSE list showing all models' RMSE
print('RMSE - AR: {0:.3f} '.format(RMSE_AR))
print('RMSE - ARMA: {0:.3f} '.format(RMSE_ARIMA))
print('RMSE - SARIMAX: {0:.3f} '.format(RMSE_SARIMAX))
print('RMSE - PROPHET: {0:.3f} '.format(RMSE_PROPHET))
print('RMSE - XGBoost: {0:.3f} '.format(RMSE_XG))
print('RMSE - Neural Network: {0:.3f} '.format(RMSE_NN))
print('RMSE - Holt-Winter: {0:.3f} '.format(RMSE_HW))
print('RMSE - Darts: {0:.3f} '.format(RMSE_darts))

RMSE_ALL=[RMSE_AR,RMSE_ARIMA,RMSE_SARIMAX,RMSE_PROPHET,RMSE_XG,RMSE_NN,RMSE_HW,RMSE_darts]


#### 33. Models' MASE comparison

In [ ]:
print('-'*5,'   MASE   ','-'*5)

#print MAPE list showing all models' RSME
print('MASE - AR: {0:.3f} '.format(MASE_AR))
print('MASE - ARMA: {0:.3f} '.format(MASE_ARIMA))
print('MASE - SARIMAX: {0:.3f} '.format(MASE_SARIMAX))
print('MASE - PROPHET: {0:.3f} '.format(MASE_PROPHET))
print('MASE - XGBoost: {0:.3f} '.format(MASE_XG))
print('MASE - Neural Network: {0:.3f} '.format(MASE_NN))
print('MASE - Holt-Winter: {0:.3f} '.format(MASE_HW))
print('MASE - Darts: {0:.3f} '.format(MASE_darts))

MASE_ALL=[MASE_AR,MASE_ARIMA,MASE_SARIMAX,MASE_PROPHET,MASE_XG,MASE_NN,MASE_HW,MASE_darts]


#### 34. Metrics Summary

In [ ]:
all_metrics=pd.DataFrame([MAE_ALL,MAPE_ALL,RMSE_ALL, MASE_ALL]).T
all_metrics.rename(columns={0:'MAE',1:'MAPE',2:'RMSE',3:'MASE'}, 
                   index={0:'AR',1:'ARIMA',2:'SARIMAX',3:'PROPHET',4:'XGBOOST',5:'RNN',6:'HOLT-WINTERS',7:'DARTS'})

<a id='FOR'></a>

# Bring forecasts back to original scale

#### 40. Monthly Forecasts

In [ ]:
# get last value of original time series 
ts_pre=ts_before_dif.rename(columns={'Values':'Original Monthly Movements'})
last_ts_value=ts_pre.iloc[-1,0]
last_ts_value

In [ ]:
# get all predictions out of sample (converted into a df)
ts_future=future_prediction_sar.to_frame().rename(columns={'predicted_mean':'Differentiated Monthly Forecasts'})
ts_future.head(2)

In [ ]:
ts_future.iloc[0,0]=last_ts_value
ts_future.head()

In [ ]:
# calculate final predictions in original values
final_predictions=ts_future.cumsum()
final_predictions=final_predictions.rename(columns={'Differentiated Monthly Forecasts':'Final Monthly Forecast'})
final_predictions.iloc[0,0]=0
final_predictions.head()

In [ ]:
# combine all data (original time series and future forecasts)
all_sales_info=pd.concat([ts_pre, ts_future,final_predictions],axis=1)
all_sales_info=all_sales_info.fillna(0)
# wrong all_sales_info['All Monthly Sales']=all_sales_info['Original Monthly Movements']+all_sales_info['Final Monthly Forecast']
all_sales_info#.tail(14)

In [ ]:
all_sales_info[['Original Monthly Movements','Final Monthly Forecast']].plot(figsize=(12,8), xlabel='Months', ylabel='£', title='Monthly Movements');

#### 41. YTD Forecast

In [ ]:
# ytd values (without future forecasts)
plt.figure(figsize=(12,8))
df_final['Original_values'].plot(figsize=(12,8), xlabel='Months', ylabel='£', title='Actuals YTD Balances £')

In [ ]:
# YTD forecast
ytd_forecasts=all_sales_info['Final Monthly Forecast'].cumsum().to_frame()
ytd_forecasts=ytd_forecasts.rename(columns={'Final Monthly Forecast':'YTD Forecast'})
ytd_forecasts.tail(14)

In [ ]:
ytd_sales_combined=pd.concat([df_final,ytd_forecasts],axis=1)
ytd_sales_combined=ytd_sales_combined.fillna(0)
ytd_sales_combined

In [ ]:
# ytd_sales_combined['Final YTD sales']=ytd_sales_combined['Original_values']+ytd_sales_combined['Final Monthly Forecast']

# ytd_sales_combined

In [ ]:
#plt.figure(figsize=(18,12))
ytd_sales_combined[['Original_values','YTD Forecast']].plot(figsize=(8,6), 
                                                            xlabel='Months', ylabel='£', title='Actuals and Forecasts -  YTD Balances £');

In [ ]:
t_end = time.time()

total_first_block = t_end-t_start

In [ ]:
total_first_block

In [ ]:
### Pickling the model file for deployment

In [ ]:
import pickle
pickle.dump(sarima_fit,open('salesmodel.pkl','wb'))
pickled_model=pickle.load(open('salesmodel.pkl','rb'))
pickled_model.predict(end=future[-1])[-len(future):]

In [ ]:
stop

<a id='Hyper'></a>

# ---------      Appendix  - Sarimax Model optimization         ------------------------

#### 50.  HyperParameters Optimization (with fixed train/ test split)

In [ ]:
# find out the data point where the train set stops and test set data start
# set 70% as train test split for running optimization below, in line with the rest of the project coding.
# Other % can be tested.

train_test_split_per_optim = train_test_split_per 
h_l_train=int(len(ts)*train_test_split_per_optim)
h_l_test= len(ts)-h_l_train

# identify the train set and test set
h_train =  train #ts[:-h_l_test] 
h_test =  test# ts[h_l_train:]

# work out shape of train and test set and ts set
print('Train: ',h_train.shape, 'Test: ', h_test.shape, 'Ts: ', ts.shape) 

In [ ]:
# Set range of p, and q to take values from 0 to 4, to be used to generate triplets and quadruplets 
# the p and q identified by arma_order_select_ic where 2 and 3 respectively so within the range of values here considered
p = q = range(0, 4)

# Set d to take values from 0 to 1.
d = range(0,1)

# Initialise a list to store the MAE, pdq and seasonal values
mae_values = []
pdq_values=[]
seasonal_values=[]

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q and no 12 (referring to the monthly nature of data)
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
# Generate a SARIMAX model with the selected list of parameters (in the itertool) for both pqd and the seasonal order
for param_pdq in pdq:# Iterate over each option of pqd
    for param_seasonal in seasonal_pdq:# Iterate over each option in seasonal_pqd        
        mod_opt=tsa.statespace.SARIMAX(h_train, trend='c',
                                                order=param_pdq, 
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)

        # Fit and predict the model
        results=mod_opt.fit()
        s_prediction=results.predict(end=ts.index[-1])[-len(test):] 
        
        # calculate MAE
        mae=mean_absolute_error(h_test, s_prediction)
        
       # add MAE and relative pdq values and seasonal values to 3 separate lists
        mae_values.append(mae)
        pdq_values.append(param_pdq)
        seasonal_values.append(param_seasonal)
        
# create a df with the 3 lists mentioned above: MAE and relative pdq values and seasonal values
S_MAE_df = pd.DataFrame(
    {'pdq_values': pdq_values,
     'seasonal_values': seasonal_values,
     'mae_values': mae_values
    })

#identify pdq and seasonal values giving lower MAE
S_MAE_df[S_MAE_df['mae_values']==S_MAE_df['mae_values'].min()]

# ------------------- cross validation with Sarimax ------------------------

#### 51. Cross Validation with Sarimax (with fixed order and seasonal order)

In [ ]:
min_train_size=int(len(ts)*.65) # set a min trainig size

# Lists which will contain all the model's train size and MAE error.

x_mae_errors = []
x_size_training = []

# Generate a SARIMAX model with the selected parameters for both pqd and the seasonal order and the train/ test split     

for training_size in range(min_train_size,len(ts) ):# Iterate over different train/ test sizes
    x_mod=tsa.statespace.SARIMAX(ts[:training_size], trend='c',
                            order=(2,0,3),
                            seasonal_order=(0,0,2,12),
                            enforce_stationarity=False,
                            enforce_invertibility=False)
    train = ts[:training_size]
    test = ts[training_size:]

    # Fit and predict the model
    x_results=x_mod.fit()
    x_prediction=x_results.predict(end=len(ts)-1)[-len(test):]
    
    # compute the MAE:
    x_mae = mean_absolute_error(ts.values[training_size:], x_prediction)
    
    # append MAE to relevant list
    x_mae_errors.append(x_mae)
    x_size_training.append(training_size)
    
# create a dataframe with training size and MAE values
all = {'Training_size':x_size_training,'MAE':x_mae_errors}
x_MAE_results = pd.DataFrame(all)
#x_MAE_results

In [ ]:
print('3 lowest MAE level are:')
x_MAE_results.sort_values(by=['MAE'])
x_MAE_results.nsmallest(3, 'MAE')
#x_MAE_results[x_MAE_results['MAE']==x_MAE_results['MAE'].min()]
#"{:.7f}".format

# --------------- Cross validation & hyper parameters  tuning -------------------

In [ ]:


t0 = time.time()

##### 52.  Cross Validation and HyperParameters Optimization (with variable train/ test split and variable order and seasonal order)

In [ ]:
# Values to generate triplets and quadruplets 
pp = qq = range(0, 4)
dd = range(0,1)

# Generate all different combinations of p, q and q triplets and quadruplets
cv_pdq = list(itertools.product(pp, dd, qq))
cv_seasonal_pdq = [(xx[0], xx[1], xx[2], 12) for xx in list(itertools.product(pp, dd, qq))]

<a id='Hyperparameter search'></a>

In [ ]:
# Lists which will contain all the model's parameters, train size and MAE error.
''' the train-test split % is set to be between 65% to 80%'''
cv_pdq_values=[]
cv_seasonal_values=[]
cv_mae_errors = []
cv_size_training = []

 # Generate a SARIMAX model with the selected parameters for both pqd and the seasonal order and the train/ test split
for cv_param_pdq in cv_pdq:# Iterate over each option of pqd
    for cv_param_seasonal in cv_seasonal_pdq:# Iterate over each option in seasonal_pqd   

        for training_size in range(min_train_size,len(ts) ):# Iterate ove each option of train/ test split
            # choose the optimal lag using AIC (this is a model selection criterion)
            cv_mod=tsa.statespace.SARIMAX(ts[:training_size], trend='c',
                                    order=cv_param_pdq,
                                    seasonal_order=cv_param_seasonal,
                                    enforce_stationarity=False,
                                    enforce_invertibility=False)
            train = ts[:training_size]
            test = ts[training_size:]

            # Fit and predict the model
            cv_results=cv_mod.fit()
            cv_prediction=cv_results.predict(end=len(ts)-1)[-len(test):] #len(h_test))
            
            # Calculate MAE
            cv_mae = mean_absolute_error(ts.values[training_size:], cv_prediction)
            
            # Append MAE, training size, order and seasonal orders values to lists
            cv_mae_errors.append(cv_mae)
            cv_size_training.append(training_size)
            cv_pdq_values.append(cv_param_pdq)
            cv_seasonal_values.append(cv_param_seasonal)
            
        #create dataframe containing all the mentioned list
        all = {'pdq':cv_pdq_values, 'Seasonal':cv_seasonal_values ,'Training_size':cv_size_training,'MAE':cv_mae_errors}
        CV_MAE_results = pd.DataFrame(all)
# show results        
CV_MAE_results

In [ ]:
CV_MAE_results.sort_values(by=['MAE'], ascending=False)

In [ ]:
print('3 lowest MAE level are:')
CV_MAE_sized=CV_MAE_results[CV_MAE_results['Training_size']<=int(ts.shape[0]*0.8)]
CV_MAE_TOP15=CV_MAE_sized.sort_values(by=['MAE'], ascending=False)
CV_MAE_TOP15.nsmallest(15, 'MAE')

In [ ]:
t1 = time.time()

total = t1-t0

In [ ]:
total